Interesting link <a href="https://yanirseroussi.com/2015/07/06/learning-about-deep-learning-through-album-cover-classification/">Album Cover CNN </a> <a href="https://github.com/yanirs/bandcamp-deep-learning/">GIT</a>
<br />
Caffe example: <a href="https://github.com/yanirs/bandcamp-deep-learning/blob/master/notebooks/02-caffe-games.ipynb">Yanirs caffe2 pretrained model</a>
<br />
<a href="https://github.com/caffe2/models">Caffe2 models</a>
<br />
<a href="https://github.com/caffe2/tutorials/blob/master/Loading_Pretrained_Models.ipynb">Loading pre-trained models in Caffe2</a>
<br/>
<a href="https://caffe2.ai/docs/tutorial-MNIST.html">CNN Caffe2 Tutorial</a>
<br />
<a href="https://towardsdatascience.com/the-softmax-function-neural-net-outputs-as-probabilities-and-ensemble-classifiers-9bd94d75932">How to use softmax output layer</a>

In [1]:
import numpy as np
from matplotlib import pyplot
import matplotlib.image as mpimg
import os
import pandas as pd
import skimage
import skimage.io as io
import skimage.transform
import operator
from urllib.request import urlopen


from caffe2.python import core, model_helper, net_drawer, workspace, visualize, brew

core.GlobalInit(['caffe2', '--caffe2_log_level=0'])

# Obtaining the Dataset

In [4]:
import requests 
import csv
from multiprocessing.pool import ThreadPool


def _download_image(args):
    session, url, local_path = args[0], args[1], args[2]
    try:
        response = session.get(url)
    except requests.RequestException:
        return False
    if response.status_code != requests.codes.ok:
        return False

    with open(local_path, 'wb') as out:
        out.write(response.content)
    return True

def download_dataset_images(out_dir, dataset_links_tsv='dataset-links.tsv', num_threads=10):
    """Download the dataset images to out_dir, based on the links in dataset_links_tsv.

    dataset_links_tsv is expected to be a tab-separated file with two columns: image url, and local path with format
    <genre>/<album_id>.jpg
    """
#     if os.path.exists(out_dir):
#         raise ValueError('%s already exists' % out_dir)

    session = requests.Session()
    with open(dataset_links_tsv) as fh:
        jobs = []
        genres = set()
        for url, rel_path in csv.reader(fh, delimiter='\t'):
            genres.add(os.path.dirname(rel_path))
            path = os.path.join(out_dir, rel_path)
            if not os.path.exists(path):
                jobs.append((session, url, os.path.join(out_dir, rel_path)))

    for genre in genres:
        genre_path = os.path.join(out_dir, genre)
        if not os.path.exists(genre_path):
            os.makedirs(genre_path)
            
    print('Still needs to download {} covers'.format(len(jobs)))
    pool = ThreadPool(num_threads)
    num_successes = sum(pool.map(_download_image, jobs))
    print('Successfully downloaded %s/%s images' % (num_successes, len(jobs)))
    
download_dataset_images('./img/')

Still needs to download 10000 covers
Successfully downloaded 9753/10000 images


# Keras Feature Extraction

### Initialize workspace

In [22]:
import numpy as np
from keras.applications import vgg16, inception_v3, resnet50, mobilenet

#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')
 
#Load the Inception_V3 model
inception_model = inception_v3.InceptionV3(weights='imagenet')
 
#Load the ResNet50 model
resnet50_model = resnet50.ResNet50(weights='imagenet')
 
#Load the ResNet50 model
resnext101_model = resnext.ResNeXt101(weights='imagenet')

#Load the MobileNet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

RuntimeError: Attempted to use a closed Session.

In [39]:
import keras_applications
img_model = keras_applications.resnext.ResNeXt101(weights = 'imagenet', 
        backend = keras.backend, layers = keras.layers, models = keras.models, utils = keras.utils)
img_app = keras_applications.resnext

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6414:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6409:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6404:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6399:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6394:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6389:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'IsVariableInitialized_6384:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3287, in run_code
    return outflag  File "<ipython-input-22-071f7e889a0e>", line 5, in <module>
    vgg_model = vgg16.VGG16(weights='imagenet')  File "/usr/lib/python3.7/site-packages/keras/applications/__init__.py", line 28, in wrapper
    return base_fun(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras/applications/vgg16.py", line 11, in VGG16
    return vgg16.VGG16(*args, **kwargs)  File "/usr/lib/python3.7/site-packages/keras_applications/vgg16.py", line 210, in VGG16
    model.load_weights(weights_path)  File "/usr/lib/python3.7/site-packages/keras/engine/network.py", line 1166, in load_weights
    f, s

Exception ignored in: <function BaseSession._Callable.__del__ at 0x7f128410ee18>
Traceback (most recent call last):
  File "/usr/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/usr/lib/python3.7/site-packages/tensorflow/python/framework/errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


178176000/178173608 [==============================] - 25s 0us/step


RuntimeError: Attempted to use a closed Session.

In [48]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
%matplotlib inline

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    K.set_session(sess)
    
    import keras_applications
    keras_applications.set_keras_submodules(backend = K)
    
    img_model = keras_applications.resnext.ResNeXt101(weights = 'imagenet', 
        backend = K, layers = keras.layers, models = keras.models, utils = keras.utils)

    df = pd.DataFrame()
    batch_size = 100
    genres = os.listdir('img')
    print(genres)
    for genre in genres:
        GENRE_LOCATION = './img/' + genre + '/'
        FOLDER = os.listdir(GENRE_LOCATION)
        for i in range(0, 1000, batch_size):
            images = [os.path.abspath(GENRE_LOCATION + f) for f in FOLDER[i:i+batch_size]]
            imgs = []
            for imgf in images:
                img = load_img(imgf, target_size=(224,224))
                x = img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = keras_applications.imagenet_utils.preprocess_input(x, mode='torch')
                imgs.append(x)
            imgs = np.concatenate(imgs, axis=0)
            print('Created batch of images.. {}: {} - {}'.format(genre, i, i + batch_size))
            img_feats = img_model.predict(imgs)
            preds = pd.DataFrame(np.squeeze(img_feats))
            preds['genre'] = genre
            df = pd.concat([df, preds])

['hiphop_rap', 'dubstep', 'rock', 'ambient', 'pop', 'punk', 'jazz', 'folk', 'soul', 'metal']
Created batch of images.. hiphop_rap: 0 - 100
Created batch of images.. hiphop_rap: 100 - 200
Created batch of images.. hiphop_rap: 200 - 300
Created batch of images.. hiphop_rap: 300 - 400
Created batch of images.. hiphop_rap: 400 - 500
Created batch of images.. hiphop_rap: 500 - 600
Created batch of images.. hiphop_rap: 600 - 700
Created batch of images.. hiphop_rap: 700 - 800
Created batch of images.. hiphop_rap: 800 - 900
Created batch of images.. hiphop_rap: 900 - 1000
Created batch of images.. dubstep: 0 - 100
Created batch of images.. dubstep: 100 - 200
Created batch of images.. dubstep: 200 - 300
Created batch of images.. dubstep: 300 - 400
Created batch of images.. dubstep: 400 - 500
Created batch of images.. dubstep: 500 - 600
Created batch of images.. dubstep: 600 - 700
Created batch of images.. dubstep: 700 - 800
Created batch of images.. dubstep: 800 - 900
Created batch of images..

In [2]:
df.to_csv('output_resnext101.csv')

NameError: name 'df' is not defined

In [39]:
# prepare the image for the VGG model
processed_image = vgg16.preprocess_input(image_batch.copy())
 
# get the predicted probabilities for each class
predictions = vgg_model.predict(processed_image)
# print predictions
 
# convert the probabilities to class labels
# We will get top 5 predictions which is the default
label = decode_predictions(predictions)
label

[[('n09229709', 'bubble', 0.07761089),
  ('n04366367', 'suspension_bridge', 0.07345834),
  ('n09428293', 'seashore', 0.058393676),
  ('n09332890', 'lakeside', 0.05620728),
  ('n03388043', 'fountain', 0.049309738)]]

In [41]:
predictions.shape

(1, 1000)

# Caffe2 Feature Extraction

### Initialize workspace

In [3]:
from caffe2.python import models
models = os.path.dirname(models.__file__)
MODEL = 'bvlc_googlenet', 'init_net.pb', 'predict_net.pb', 'ilsvrc_2012_mean.npy', 227
IMAGE_LOCATION = "img/ambient/a4072966.jpg"
codes =  "https://gist.githubusercontent.com/aaronmarkham/cd3a6b6ac071eca6f7b4a6e40e6038aa/raw/9edb4038a37da6b5a44c3b5bc52e448ff09bfe5b/alexnet_codes"

MEAN_FILE = os.path.join(models, MODEL[0], MODEL[3])
if not os.path.exists(MEAN_FILE):
    print("No mean file found!")
    mean = 128
else:
    print ("Mean file found!")
    mean = np.load(MEAN_FILE).mean(1).mean(1)
    mean = mean[:, np.newaxis, np.newaxis]
print("mean was set to: ", mean)

INIT_NET = os.path.join(models, MODEL[0], MODEL[1])
PREDICT_NET = os.path.join(models, MODEL[0], MODEL[2])
print('Init net file: ', INIT_NET)
print('Predict net file: ', PREDICT_NET)

# Read predictor required init, predict files
with open(INIT_NET, "rb") as f:
    init_net = f.read()
with open(PREDICT_NET, "rb") as f:
    predict_net = f.read()
    
# Initialize the workspace predictor
p = workspace.Predictor(init_net, predict_net)

# Set the labels in a class array
with open('codes.txt', 'r') as response:
    class_LUT = []
    for line in response:
        code, result = line.partition(":")[::2]
        code = code.strip()
        result = result.replace("'", "")
        if code.isdigit():
            class_LUT.append(result.split(",")[0][1:])
            


No mean file found!
mean was set to:  128
Init net file:  /usr/lib/python3.7/site-packages/caffe2/python/models/bvlc_googlenet/init_net.pb
Predict net file:  /usr/lib/python3.7/site-packages/caffe2/python/models/bvlc_googlenet/predict_net.pb


### Add functions

In [4]:
def crop_center(img,cropx,cropy):
    y,x,c = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

def rescale(img_location, input_height, input_width):
    img = skimage.data.load(img_location)
    aspect = img.shape[1]/float(img.shape[0])

    if aspect > 1:
        # landscape orientation - wide image
        res = int(aspect * input_height)
        imgScaled = skimage.transform.resize(img, (input_width, res))
    elif aspect < 1:
        # portrait orientation - tall image
        res = int(input_width/aspect)
        imgScaled = skimage.transform.resize(img, (res, input_height))
    elif aspect == 1:
        imgScaled = skimage.transform.resize(img, (input_width, input_height))
    
    return imgScaled

# Initialize batch array
def create_img_batch(images):
    height, width = 224, 224
    NCHW_batch = np.zeros((len(images),3,height,width))
    # For each of the images in the list, format it and place it in the batch
    for i, curr_img in enumerate(images):
        img = rescale(curr_img, height, width)
        img = crop_center(img, height, width)
        img = img.swapaxes(1, 2).swapaxes(0, 1)
        img = img[(2, 1, 0), :, :]
        img = img * 255 - mean
        NCHW_batch[i] = img
    print ("Batch Shape: ",NCHW_batch.shape)
    return NCHW_batch

#Parsing the results of one image, creating topN predictions
def parse_results(results, topN):
    index = 0
    highest = 0
    arr = np.empty((0,2), dtype=object)
    arr[:,0] = int(10)
    arr[:,1:] = float(10)
    for i, r in enumerate(results):
        # imagenet index begins with 1!
        i=i
        arr = np.append(arr, np.array([[i,r]]), axis=0)
        if (r > highest):
            highest = r
            index = i 
            
    topN = sorted(arr, key=lambda x: x[1], reverse=True)[:topN]
#     print("Raw top {} results: {}".format(N,topN))
    prediction = []
    for i, p in topN:
        pred = (class_LUT[int(i)], round(p * 100, 2))
        prediction.append(pred)
    return prediction

def get_labels_for_images(images):
    """Returns list of labels that were predicted from given images"""
    batch = create_img_batch(images)
    results = p.run([batch.astype(np.float32)])

    # Turn it into a multi-dimensional array
    results = np.asarray(results)
    # Squeeze out the unnecessary axis
    preds = np.squeeze(results)
    print("Squeezed Predictions Shape, with batch size {}: {}".format(len(images),preds.shape))

    labels = []
    for i, pred in enumerate(preds):
        predictions = parse_results(pred, 10)
#         print('Predictions for img: {}:'.format(images[i]))
        for pred in predictions:
#             print(pred)
            labels.append(pred[0])
    return labels

def add_labels_to_genre(labels, genre):
    for word in punk:
        genre[word] = genre.get(word, 0) + 1
    return genre


## Create output dataframe

In [40]:
df = pd.DataFrame()
batch_size = 100
genres = {f:{} for f in os.listdir('img')}
for genre in genres:
    GENRE_LOCATION = './img/' + genre + '/'
    FOLDER = os.listdir(GENRE_LOCATION)
    for i in range(0, 1000, batch_size):
        images = [os.path.abspath(GENRE_LOCATION + f) for f in FOLDER[i:i+batch_size]]
        batch = create_img_batch(images)
        results = p.run([batch.astype(np.float32)])
        # Turn it into a multi-dimensional array
        results = np.asarray(results)
        # Squeeze out the unnecessary axis
        preds = pd.DataFrame(np.squeeze(results))
        preds['genre'] = genre
        df = pd.concat([df, preds])
# # Describe the results
# for i, pred in enumerate(preds):
#     print("Results for: '{}' :{}".format(images[i], pred))
#     # Get the prediction and the confidence by finding the maximum value 
#     #   and index of maximum value in preds array
#     curr_pred, curr_conf = max(enumerate(pred), key=operator.itemgetter(1))
#     print("\tPrediction: ", curr_pred)
#     print("\tClass Name: ", class_LUT[int(curr_pred)])
#     print("\tConfidence: ", curr_conf)

Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (86, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (79, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (100, 3, 224, 224)
Batch Shape:  (76, 3, 224, 224)
Batch Shape: 

# Network

In [18]:
import pandas as pd
df = pd.read_csv('output_resnet.csv')

In [19]:
totaal = len(df)
values = df.genre.value_counts()
print(totaal)
for v in values:
    print(v, (v / totaal * 100))

9753
986 10.109709832871937
982 10.068696811237569
981 10.058443555828976
979 10.037937045011791
976 10.007177278786013
975 9.996924023377423
974 9.98667076796883
973 9.976417512560237
971 9.955911001743054
956 9.802112170614171


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split

df2 = df#df[df.genre.isin(['metal', 'folk', 'jazz'])]
# 10 Genres
n_classes = len(df2.genre.unique())
Y = df2.genre
X = df2.iloc[:, 0:1000]
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
labels = list(encoder.inverse_transform(range(0,n_classes)))
# encoded_Y = keras.utils.to_categorical(encoded_Y)
binarized_Y = label_binarize(encoded_Y, classes=range(0,n_classes))
X_train, X_test, Y_train, Y_test = train_test_split(X, binarized_Y, test_size=0.33)
labels
n_classes

In [42]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV


def get_model(genres=10, dropout_1=0.5, output_1=256, activation_1='relu',
             dropout_2=0.5, four=False):
    
    model = Sequential()
    # Model has an input_dimension of 1000 (all the words)
    model.add(Dense(1000, input_dim=1000, activation='relu'))
    model.add(Dropout(dropout_1))      
    model.add(Dense(output_1))
    model.add(Activation(activation_1))
    model.add(Dropout(dropout_2))

    #Optimizing the number of layers
    if four:
        model.add(Dense(100, input_dim=output_1, activation='relu'))
        #Choosing between complete sets of layers
        model.add(Dropout(0.5, activation='linear'))
        
    # Adding the last dense softmax layer
    model.add(Dense(genres, activation='softmax' if genres > 2 else 'sigmoid'))
    adm = Adam(lr=0.001, beta_1=0.7, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy' if n_classes > 2 else 'binary_crossentropy',
                  optimizer=adm,
                  metrics=['accuracy'])
    return model

k_model = KerasClassifier(build_fn=get_model, verbose=0)
# model = get_model(n_classes if n_classes > 2 else n_classes - 1)


# grid search epochs, batch size
epochs = [1, 10] # add 50, 100, 150 etc
batch_size = [10, 64, 256, 512, 1000] # add 5, 10, 20, 40, 60, 80, 100 etc
param_grid = dict(epochs=epochs, batch_size=batch_size)

grid = GridSearchCV(estimator=k_model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, Y_train) 

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# result = model.fit(X_train, Y_train,
#           epochs=20,
#           batch_size=128)

# score = model.evaluate(X_test, Y_test, batch_size=128)

/usr/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.105278 using {'batch_size': 1000, 'epochs': 1}
0.100492 (0.000670) with: {'batch_size': 10, 'epochs': 1}
0.102543 (0.002091) with: {'batch_size': 10, 'epochs': 10}
0.100219 (0.007279) with: {'batch_size': 64, 'epochs': 1}
0.102133 (0.001675) with: {'batch_size': 64, 'epochs': 10}
0.093656 (0.002352) with: {'batch_size': 256, 'epochs': 1}
0.098168 (0.004497) with: {'batch_size': 256, 'epochs': 10}
0.102953 (0.004276) with: {'batch_size': 512, 'epochs': 1}
0.098988 (0.006178) with: {'batch_size': 512, 'epochs': 10}
0.105278 (0.001583) with: {'batch_size': 1000, 'epochs': 1}
0.099672 (0.005057) with: {'batch_size': 1000, 'epochs': 10}


In [22]:
score = model.evaluate(X_test, Y_test, batch_size=128)
score

487/487 [==============================] - 0s 45us/step


[0.6904220882137698, 0.5441478460231601]

In [23]:
y_score = model.predict(X_test, batch_size=128)
encoder.inverse_transform([np.argmax(y_score[0]), np.argmax(Y_test[0])])

NameError: name 'np' is not defined

In [26]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(labels[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()


plt.figure(2)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(labels[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()


IndexError: index 1 is out of bounds for axis 1 with size 1

### Test image conversion

In [34]:


img, imgBGR = get_resized(TEST_IMG, 224, 224)
pyplot.figure()
pyplot.subplot(1,2,1)
pyplot.imshow(img)
pyplot.axis('on')
pyplot.title('Original image = RGB')

pyplot.figure()
for i in range(3):
    # For some reason, pyplot subplot follows Matlab's indexing
    # convention (starting with 1). Well, we'll just follow it...
    pyplot.subplot(1, 3, i+1)
    pyplot.imshow(imgBGR[i])
    pyplot.axis('off')
    pyplot.title('BGR channel %d' % (i+1))
    
imgCropped = imgBGR[np.newaxis, :, :, :].astype(np.float32)
print('Final input shape is:', imgCropped.shape)

NameError: name 'get_resized' is not defined

## Look through the downloaded Albums

In [69]:
class Album:
    
    def __init__(self, path, infos):
        self.path = path
        self.name = infos[0]
        self.genre = infos[1]
        self.label = infos[2]
        self.year = infos[3]

def read_files(file_list):
    albums = []
    fucked = []
    for i in range(len(file_list)):
        file_name = file_list[i]
        split = file_name.split('-')
        if file_name.count('-') >= 4:
            print('FUCK: ', file_name)
            fucked.append(file_name)
        else:
            albums.append(Album(os.path.abspath(file_name), split))
    return albums, fucked
    

def explore_files():
    current_folder_filename_list = os.listdir("./data") # Change this PATH to traverse other directories if you want.
    albums, fucked = [], []
    if current_folder_filename_list != None:
        print("%s files were found under current folder. " % len(current_folder_filename_list))
        albums, fucked = read_files(current_folder_filename_list)
    return albums, fucked

albums, fucked = explore_files()
genres = [a.genre for a in albums]

unique_genres = sorted(set(genres))
print("We start with {} genres which are definitely too many labels for a CNN at this point.".format(len(unique_genres)))

4064 files were found under current folder. 
FUCK:  TM404-Dub Techno-Kontra-Musik-2013.jpg
FUCK:  Knee-Deep In The North Sea-Jazz-Real World-2007.jpg
FUCK:  ZGMF-X19A-UK Bass-Infinite Machine-2015.jpg
FUCK:  Singles 93-03-Big Beat-Virgin-2003.jpg
FUCK:  Everywhere at the end of time - Stage 4-Dark Ambient-History Always Favours The Winners-2018.jpg
FUCK:  Acquiescence-Downtempo-Non-Projects-2012.jpg
FUCK:  32 Zel  Planet Shrooms-Hip-Hop--2015.jpg
FUCK:  Jenny Death-Hip-Hop-Harvest Records-2015.jpg
FUCK:  Re-Animations EP-Footwork-Astrophonica-2013.jpg
FUCK:  Czarface meets Metal Face-Hip-Hop-Silver Age-2018.jpg
FUCK:  And. IV [Inertia]-Glitch-Raster-Noton-2012.jpg
FUCK:  EP1 Drum Talking-Dub-One-Handed Music-2011.jpg
FUCK:  Alas Rattoisaa Virtaa-Neo-Psychedelia-Dekorder-2014.jpg
FUCK:  Old-Hip-Hop-Fool's Gold Records-2013.jpg
FUCK:  Inner Treaty-Neo-Psychedelia-Sun Ark Records-2012.jpg
FUCK:  volume 1 flick your tongue against your teeth and describe the present.-Hip-Hop--2017.png
FUCK

# Clarifai Analysis

In [9]:
from clarifai.rest import ClarifaiApp
app = ClarifaiApp()

m = app.models.get('aaa03c23b3724a16a56b629203edc62c')

# app.tag_images('http://assets.rollingstone.com/assets/images/list/0e099b2214b1673fc76c6c60257b88aefe571def.jpg')

In [30]:
def get_albums_img(main_folder):
    img_folder_list = os.listdir(main_folder) # Change this PATH to traverse other directories if you want.
    albums = {}
    for folder in img_folder_list:
        print(folder)
        if folder not in albums.keys():
            albums[folder] = []
        # Retrieve images for this album
        images = os.listdir(main_folder + '/' +folder)
        for image in images:
            if image not in albums[folder]:
                albums[folder].append(image)
                
    return albums

albums = get_albums_img("./img")

def get_absolute_image_path(genre, image):
    return os.path.abspath("./img/{}/{}".format(genre, image))

hiphop_rap
dubstep
rock
ambient
pop
punk
jazz
folk
soul
metal


In [32]:
tryout = albums['ambient'][0:1]
for img in tryout:
    path = get_absolute_image_path('ambient', img)
    tags = m.predict('http://assets.rollingstone.com/assets/images/list/0e099b2214b1673fc76c6c60257b88aefe571def.jpg')

/home/krulvis/PycharmProjects/AlbumAnalyser/img/ambient/a1604573766.jpg
